In [ ]:
import os
from functools import reduce
from glob import glob

import pandas as pd

In [ ]:
root_dir = "/home/xultaeculcis/Downloads/logs"
csvs = sorted(glob(os.path.join(root_dir, "*.csv")))

frames = []
for fp in csvs:
    fname = os.path.basename(fp).replace(".csv", "")
    metric = fname.split("tag")[-1].strip("-")
    var = fname.split("-")[5]
    metric = metric.replace("val_", "").replace("train_", "")

    if var == "temp":
        continue

    df = pd.read_csv(fp)
    df[f"{var}/{metric}"] = df["Value"]
    df = df.drop(["Value", "Wall time"], axis=1)
    df.columns = [c.lower() for c in df.columns]
    frames.append(df)

frame = reduce(
    lambda left, right: pd.merge(left, right, how="left", on=["step"]), frames
)
frame.to_csv("../app/assets/climate-sr/metrics.csv", index=False)

In [ ]:
root_dir = "/home/xultaeculcis/Downloads/logs"
csvs = sorted(glob(os.path.join(root_dir, "*.csv")))

frames = []
for fp in csvs:
    fname = os.path.basename(fp).replace(".csv", "")
    metric = fname.split("tag")[-1].strip("-")
    var = fname.split("-")[5]
    metric = metric.replace("val_", "").replace("train_", "")

    if var in ["tmin", "tmax", "prec"]:
        continue

    df = pd.read_csv(fp)
    df[f"{var}/{metric}"] = df["Value"]
    df = df.drop(["Value", "Wall time"], axis=1)
    df.columns = [c.lower() for c in df.columns]
    frames.append(df)

frame = reduce(
    lambda left, right: pd.merge(left, right, how="left", on=["step"]), frames
)
frame.to_csv("../app/assets/climate-sr/metrics-temp.csv", index=False)

In [ ]:
df1 = pd.read_parquet("../app/assets/climate-sr/metrics-temp.parquet")
df2 = pd.read_parquet("../app/assets/climate-sr/metrics.parquet")

pd.merge(df1, df2, on="step", how="outer").sort_values(by="step").interpolate().fillna(
    method="bfill"
).to_csv("../app/assets/climate-sr/metrics.csv", index=False)